# Lecture 19: Convolutional Neural Networks in Keras
***

![digits](figs/mnist.png "mnist data")


***

In this notebook we'll experiment with using Keras and convolutional neural networks to classify handwritten digits from the MNIST data set.  

### Problem 1: Setup and Data Exploration  
***

If you do not have Keras installed on your machine, you can do this quickly and easily.  

In MacOS or Linux you can install Keras using 

In [ ]:
conda install -c conda-forge keras 

If you're running Windows then you can install Keras from the `conda prompt` with 

In [ ]:
conda install --channel https://conda.anaconda.org/conda-forge keras

**Part A**: First, we'll load the MNIST data directly from Keras. 

In [ ]:
from keras.datasets import mnist

(X_train, y_train), (X_valid, y_valid) = mnist.load_data()

Note that the data is already partitioned into training and validation sets.  Because training CNNs is a time-intensive process, we'll reduce the size of both data sets so that they're more manageable. 

In [ ]:
num_train, num_valid = 5000, 1000

X_train = X_train[:num_train]
y_train = y_train[:num_train]
X_valid = X_valid[:num_valid]
y_valid = y_valid[:num_valid]

**Q**: What are the shape and dimensions of the data? 

**Part B**: When working with CNNs we need to reshape our data to conform to general image types, which typically have multiple color bands.  We need to reshape the data arrays to have 4 dimensions, where the additional data corresponds to a single color band.  How we do this will depend on whether we're using `Theano` or `TensorFlow` as the Keras backed.  

In [ ]:
from keras import backend as K

# set the size of the images 
img_rows, img_cols = 28, 28

# Add the color-band dimension either before or after the image dimensions 
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_valid = X_valid.reshape(X_valid.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_valid = X_valid.reshape(X_valid.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
print("new data dimension: ", X_train.shape)

We'll also normalize the pixel data so that intensities fall between $[0,1]$ instead of $[0,255]$. 

In [ ]:
X_train = X_train.astype('float32') / 255.0 
X_valid = X_valid.astype('float32') / 255.0 

**Part C**: Finally, we need to modify the data labels so that they correspond to one-hot-encoded labels.  Luckily, Keras has a function to do this for us automatically. 

In [ ]:
import numpy as np

from keras.utils import np_utils 

num_classes = len(np.unique(y_train))

y_train_onehot = np_utils.to_categorical(y_train, num_classes) 
y_valid_onehot = np_utils.to_categorical(y_valid, num_classes) 

print("example one-hot-encoded labels: ")
for ii in range(3):
    print("digit label: {:d}, one-hot-label: {}".format(y_train[ii], y_train_onehot[ii]))

**Part D**: Finally, let's look at a few of the images, just to make sure we haven't messed anything up.

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(6,3))
for ii, ax in enumerate(axes):
    if K.image_data_format() == 'channels_first':
        ax.imshow(X_train[ii,0,:,:], cmap='gray')
    else:
        ax.imshow(X_train[ii,:,:,0], cmap='gray')
    ax.set_title("Ground Truth : {:d}".format(y_train[ii]))
    ax.axis('off')

### Problem 2: Our First Simple CNN
***

**Part A**: We will train a CNN with the following architecture.  Note that we're just working from good starting points here.  In practice we would want to spend time analyzing and evaluating multiple architectures, activations, and hyperparameters. 

Our CNN will be comprised of 

- 3 Convolutional Layers of increasing number of filters, each followed by a MaxPool Layer 
- A Dense hidden layer with 128 neurons and a linear activation function 
- An output layer with softmax activation 

First, we need to load all of the necessary modules from Keras 

In [ ]:
import keras

from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

Additionally, we need to define certain hyperparameter like batch_size, number of epochs, etc

In [ ]:
batch_size = 64 
num_epochs = 20 

Finally, we initialize a model and start adding layers 

In [ ]:
# initialize model 
simple_model = Sequential()

# Convolutional layer with 32 3x3 filters followed by 2x2 MaxPool 
simple_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape,padding='same'))
simple_model.add(MaxPooling2D((2, 2),padding='same'))

# Convolutional layer with 64 3x3 filters followed by 2x2 MaxPool 
simple_model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
simple_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

# Convolutional layer with 128 3x3 filters followed by 2x2 MaxPool 
simple_model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
simple_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

# Flatten and feed into dense layer 
simple_model.add(Flatten())
simple_model.add(Dense(128, activation='relu'))

# Output layer 
simple_model.add(Dense(num_classes, activation='softmax'))

Finally, we have to specify the loss function and solver we'll use during training, as well as the metric that we want reported to us throughout the process. 

In [ ]:
loss = keras.losses.categorical_crossentropy
optimizer = keras.optimizers.Adam()
metrics = ["accuracy"]

And now we tell Keras to compile the model

In [ ]:
simple_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

Once our model is set, Keras can print a nice summary for us 

In [ ]:
simple_model.summary()

**Part B**: Now we're ready to actually train the model! You might want to grab a magazine or something because this'll take a minute!   

In [ ]:
simple_train = simple_model.fit(X_train, y_train_onehot, batch_size=batch_size, epochs=num_epochs, verbose=1,
                                validation_data=(X_valid, y_valid_onehot))

**Part C**: We can already get an idea of how we did from the training output, but let's get the final results directly from the training history and print them. 

In [ ]:
print("training loss = {:.3f},  training accuracy = {:.3f}".
      format(simple_train.history['loss'][-1], simple_train.history['acc'][-1]))

We can also print the loss and accuracy on the validation data. 

In [ ]:
print("validation loss = {:.3f},  validation accuracy = {:.3f}".
      format(simple_train.history['val_loss'][-1], simple_train.history['val_acc'][-1]))

**Part D**: Finally, let's make a plot of the training and validation accuracies as they evolve over the training process. 

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
ax.plot(range(1,num_epochs+1), simple_train.history['acc'], color="steelblue", marker="o", label="training")
ax.plot(range(1,num_epochs+1), simple_train.history['val_acc'], color="green", marker="o", label="validation")
ax.grid(alpha=0.25)
ax.legend(loc="lower right", fontsize=20)
plt.xticks(range(1,num_epochs+2,2))
ax.set_xlabel("epoch", fontsize=16)
ax.set_ylabel("accuracy", fontsize=16)
;

### Problem 3: Combating Overfitting 
*** 

While it doesn't appear that we really have an overfitting problem here, let's pretend that we do so we can explore methods to mitigate it.  

We'll do this by adding Dropout layers after each of the Conv-MaxPool layers and after the Dense hidden layer.  You can do this by inserting layers of the form 

`simple_model.add(Dropout(p))`

in the architecture specifications above. The parameter $p$ indicates what proportions of units to randomly ignore during training.  Typically we start with small values of $p$ and then increase them as we go farther into the network.  As a first pass, try using $p = 0.25, 0.25, 0.4, 0.3$ for the four DropOut layers. 

Repeat the training and evaluation process performed in **Problem 2**.  How did you do? 

### Problem 4: More Interesting Data 
*** 

As people became board with the vanilla MNIST data, lots of alternative data sets popped up.  A popular one is the so called `fashion_mnist` dataset.  Go back to **Problem 2** and try your network out on this data set instead. 